In [1]:
pip install tensorflow==2.0.0

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_file = "/content/drive/My Drive/dataset/new.zip"
w_file = "/content/drive/My Drive/dataset/weights/"

In [0]:
import zipfile

with zipfile.ZipFile(train_file, 'r') as z:
    z.extractall()

In [0]:
import os
import cv2 as cv

def load_image(file_path):
    return cv.imread(file_path)

def extract_label(file_name):
    if "030_" in file_name:
        return 0
    elif "040_" in file_name:
        return 1
    elif "060_" in file_name:
        return 2
    elif "080_" in file_name:
        return 3
    else: return 4

train_path = "./new/"

training_files = os.listdir(train_path)

train_images = [load_image(train_path + file) for file in training_files]
train_labels = [extract_label(file) for file in training_files]

In [0]:
#os.listdir("./RenameTrain")

In [0]:
#import matplotlib.pyplot as plt
#%matplotlib inline

#preview_index = 7
#plt.subplot(1,2,1)
#plt.imshow(train_images[preview_index])
#plt.subplot(1,2,2)
#plt.imshow(preprocess_image(train_images[preview_index]))

In [0]:
def preprocess_image(img, side=50):
    min_side = min(img.shape[0], img.shape[1])
    img = img[:min_side, :min_side]
    img = cv.resize(img, (side,side))
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    return img / 255.0

In [0]:
for i in range(len(train_images)):
      train_images[i] = preprocess_image(train_images[i])

In [10]:
import numpy as np

train_images = np.expand_dims(train_images, axis=-1)
train_labels = np.array(train_labels)
print(train_images.shape, train_labels.shape)

(1078, 50, 50, 1) (1078,)


In [11]:
import tensorflow as tf

num_classes = 5
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes, dtype='float32')
print(train_labels.shape)

(1078, 5)


In [12]:
print("Tensorflow:", tf.__version__)

layers = [
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), padding="same", activation=tf.nn.relu, input_shape=train_images.shape[1:]),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Dropout(0.5),
    #tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation=tf.nn.relu),
    #tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=5, activation=tf.nn.softmax)
]

model = tf.keras.Sequential(layers)
model.compile(optimizer=tf.optimizers.Adam(),
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=[tf.metrics.CategoricalAccuracy()])
model.fit(train_images, train_labels, shuffle=True, epochs=8, batch_size=8)
#model.save_weights(w_file + "weight91.tf")

Tensorflow: 2.0.0
Train on 1078 samples
Epoch 1/8
1078/1078 [==============================] - 3s 3ms/sample - loss: 1.5734 - categorical_accuracy: 0.2579
Epoch 2/8
1078/1078 [==============================] - 3s 2ms/sample - loss: 1.3056 - categorical_accuracy: 0.4935
Epoch 3/8
1078/1078 [==============================] - 3s 3ms/sample - loss: 0.8220 - categorical_accuracy: 0.7087
Epoch 4/8
1078/1078 [==============================] - 3s 2ms/sample - loss: 0.5387 - categorical_accuracy: 0.8173
Epoch 5/8
1078/1078 [==============================] - 3s 2ms/sample - loss: 0.3870 - categorical_accuracy: 0.8720
Epoch 6/8
1078/1078 [==============================] - 3s 2ms/sample - loss: 0.4002 - categorical_accuracy: 0.8609
Epoch 7/8
1078/1078 [==============================] - 3s 2ms/sample - loss: 0.2696 - categorical_accuracy: 0.9128
Epoch 8/8
1078/1078 [==============================] - 3s 2ms/sample - loss: 0.2766 - categorical_accuracy: 0.9054


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 50, 50, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [14]:
!rm *.png *.jpg

rm: cannot remove '*.png': No such file or directory
rm: cannot remove '*.jpg': No such file or directory


In [15]:
from google.colab import files
uploads = files.upload()

MessageError: ignored

In [0]:
#Score trained model.
#scores = model.evaluate(train_images, train_labels, verbose=1)
#print('Test loss:', scores[0])
#print('Test accuracy:', scores[1])

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

classes = ['30km/h','40km/h','60km/h','80km/h','not']
eval_images = [preprocess_image(load_image(file)) for file in uploads.keys()]
eval_model = tf.keras.Sequential(layers)
eval_model.load_weights(w_file + "weight1.tf")
eval_predictions = eval_model.predict(np.expand_dims(eval_images, axis=-1))

cols = 5
rows = np.ceil(len(eval_images)/cols)
fig = plt.gcf()
fig.set_size_inches(cols * 3, rows * 3)
for i in range(len(eval_images)):
    plt.subplot(rows, cols, i+1)
    plt.imshow(eval_images[i], cmap="gray")
    plt.title(classes[np.argmax(eval_predictions[i])], color='r')
    plt.axis('off')

Reference:

https://github.com/the-computer-scientist/TensorflowTutorials/blob/master/CatsOrDogsClassification.ipynb